In [1]:
# pip install openpyxl instalar esto
import rdata
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import statsmodels.tsa as tsa
import statsmodels as sm
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy.stats import shapiro

In [2]:
xls = pd.read_excel("./data/preciosLimpio.xlsx", sheet_name=None)

resultados = {}

for nombre_hoja, df in xls.items():
    hoja_df = pd.read_excel("./data/preciosLimpio.xlsx", sheet_name=nombre_hoja, header=7, usecols="A,C,D,E,F,G,H")
    resultados[nombre_hoja] = hoja_df

In [3]:
for nombre_hoja in resultados:
    df = resultados[nombre_hoja]
    resultados[nombre_hoja] = df.iloc[1:].reset_index(drop=True)

In [4]:
df_final = pd.DataFrame()

for nombre_hoja, df in resultados.items():
    df["Año"] = nombre_hoja
    df_final = pd.concat([df_final, df], ignore_index=True)

In [5]:
df_final.drop(columns=["Bunker"], inplace=True)
df_final.set_index("FECHA", inplace=True)
df_final.sort_index(ascending=True, inplace=True)

In [19]:
df_final.index = pd.to_datetime(df_final.index)

# Filtrar test: todo el año 2025
test_gas = df_final[df_final.index.year == 2025]

# Filtrar train: todo antes de 2025
train_gas = df_final[df_final.index.year < 2025]


In [17]:
train_gas["Año"]

FECHA
2021-01-01    2021
2021-01-02    2021
2021-01-03    2021
2021-01-04    2021
2021-01-05    2021
              ... 
2024-12-27    2024
2024-12-28    2024
2024-12-29    2024
2024-12-30    2024
2024-12-31    2024
Freq: D, Name: Año, Length: 1461, dtype: object

In [18]:
# Asegurar que el índice es datetime
train_gas.index = pd.to_datetime(train_gas.index)

# Ver inicio y fin
print("Inicio:", train_gas.index.min())
print("Fin:", train_gas.index.max())

# Verificar frecuencia estimada
frecuencia = pd.infer_freq(train_gas.index)
print("Frecuencia estimada:", frecuencia)

# Si no tiene frecuencia definida, puedes asignarla si sabes que es diaria
train_gas = train_gas.asfreq('D')


Inicio: 2021-01-01 00:00:00
Fin: 2024-12-31 00:00:00
Frecuencia estimada: D
